<a href="https://colab.research.google.com/github/Srabani13/Forgotten_Florida_Race_50_miles/blob/main/FF_Race_50mile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1.Combine first and last name
#2.Create total minutes
#3.Rename division to Gender
#4.Need to put city state in lat long

In [ ]:
!pip install geopy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Importing Nominatim geocoder from the geopy library for converting addresses into latitude and longitude (geocoding)
from geopy.geocoders import Nominatim
# Importing RateLimiter from geopy to manage the rate of requests to the geocoding service (prevent hitting usage limits)
from geopy.extra.rate_limiter import RateLimiter
# Importing GeocoderTimedOut exception to handle timeout errors that might occur during geocoding
from geopy.exc import GeocoderTimedOut

In [ ]:
from google.colab import files
uploaded = files.upload()  #This will allow me to upload a file directly from my local machine


Saving Forgotten_Florida_Race_50.csv to Forgotten_Florida_Race_50 (1).csv


In [ ]:
df = pd.read_csv('Forgotten_Florida_Race_50.csv')


In [ ]:
df.head(4)


,Place,First,Last,City,State,Age,Division,Time,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1,Daniel,Wilson,Tulsa,OK,35,M,08:23:01,NaN,NaN,NaN
1,2,Eric,Davis,Greenwood,IN,38,M,08:57:54,NaN,NaN,NaN
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,09:24:35,NaN,NaN,NaN
3,4,Ron,Hammett,Montverde,FL,53,M,09:24:36,NaN,NaN,NaN


In [ ]:
df_race_results = df.dropna(axis=1)

In [ ]:
df_race_results.head(4)


,Place,First,Last,City,State,Age,Division,Time
0,1,Daniel,Wilson,Tulsa,OK,35,M,08:23:01
1,2,Eric,Davis,Greenwood,IN,38,M,08:57:54
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,09:24:35
3,4,Ron,Hammett,Montverde,FL,53,M,09:24:36


In [ ]:
df_race_results['Full Name']= df_race_results['First'] + ' ' + df_race_results['Last']
df_race_results['Full Name']

<ipython-input-64-2e3db4cc1128>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_race_results['Full Name']= df_race_results['First'] + ' ' + df_race_results['Last']


,Full Name
0,Daniel Wilson
1,Eric Davis
2,Stewart Edwards
3,Ron Hammett
4,Seth Cain
...,...
104,David Buning
105,Kelly Stoner Baker
106,Chad Ward
107,Dennis Stadelman


In [ ]:
## Rearrange the columns position
df_race_results = df_race_results[['Place', 'First', 'Last','Full Name','City', 'State', 'Age', 'Division','Time']]

In [ ]:
df_race_results.head(4)

,Place,First,Last,Full Name,City,State,Age,Division,Time
0,1,Daniel,Wilson,Daniel Wilson,Tulsa,OK,35,M,08:23:01
1,2,Eric,Davis,Eric Davis,Greenwood,IN,38,M,08:57:54
2,3,Stewart,Edwards,Stewart Edwards,New Smyrna Beach,FL,43,M,09:24:35
3,4,Ron,Hammett,Ron Hammett,Montverde,FL,53,M,09:24:36


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_race_results)

https://docs.google.com/spreadsheets/d/1XOb2h1dtvm8luvfJIqk-cPqS6DAJDGDaP4BiriVLjwU#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})


In [ ]:
# Check the data type of the 'Time' column
print(df_race_results['Time'].dtype)
# Check the first few rows to see the content
print(df_race_results['Time'].head())


object
0    08:23:01
1    08:57:54
2    09:24:35
3    09:24:36
4    09:42:17
Name: Time, dtype: object


In [ ]:
# Convert the 'Time' column to string first
df_race_results['Time'] = df_race_results['Time'].astype(str)


In [ ]:
# Convert 'Time' to timedelta
df_race_results['Time'] = pd.to_timedelta(df_race_results['Time'], errors='coerce')

In [ ]:
# Calculate total minutes
df_race_results['Total_mints'] = df_race_results['Time'].dt.total_seconds() / 60

# Convert to integer and round if necessary
df_race_results['Total_mints'] = df_race_results['Total_mints'].round().astype(int)

# Display the updated DataFrame
df_race_results.head(4)



,Place,First,Last,Full Name,City,State,Age,Division,Time,Total_mints
0,1,Daniel,Wilson,Daniel Wilson,Tulsa,OK,35,M,0 days 08:23:01,503
1,2,Eric,Davis,Eric Davis,Greenwood,IN,38,M,0 days 08:57:54,538
2,3,Stewart,Edwards,Stewart Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,565
3,4,Ron,Hammett,Ron Hammett,Montverde,FL,53,M,0 days 09:24:36,565


In [ ]:
df_race_results.rename(columns = {'Division':'Gender'}, inplace =True )

In [ ]:
def get_lat_long(city, state):
    # Combines city and state into a single address string
    address = f"{city}, {state}"

    try:
        # Creates a geolocator object that will be used to find the location of the address
        geolocator = Nominatim(user_agent="running", timeout=10)

        # Uses the geolocator to search for the address and get the location data (latitude and longitude)
        location = geolocator.geocode(address)

        # If a location is found, return the latitude and longitude
        if location:
            return location.latitude, location.longitude

        # If the location is not found, return None for both latitude and longitude
        else:
            return None, None

    # If the geolocator takes too long or times out, return None for both values
    except GeocoderTimedOut:
        return None, None


In [ ]:
df_race_results['latitude'], df_race_results['longitude'] = zip(*df_race_results.apply(lambda row: get_lat_long(row['City'], row['State']), axis=1))

In [ ]:
df_race_results.head(4)

,Place,First,Last,Full Name,City,State,Age,Gender,Time,Total_mints,latitude,longitude
0,1,Daniel,Wilson,Daniel Wilson,Tulsa,OK,35,M,0 days 08:23:01,503,36.156312,-95.992752
1,2,Eric,Davis,Eric Davis,Greenwood,IN,38,M,0 days 08:57:54,538,39.613699,-86.109543
2,3,Stewart,Edwards,Stewart Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,565,29.025813,-80.927127
3,4,Ron,Hammett,Ron Hammett,Montverde,FL,53,M,0 days 09:24:36,565,28.600277,-81.673964


In [ ]:
df_race_results['latlong']= df_race_results['latitude'].astype(str) + ', ' + df_race_results['longitude'].astype(str)
df_race_results.head(2)

,Place,First,Last,Full Name,City,State,Age,Gender,Time,Total_mints,latitude,longitude,latlong
0,1,Daniel,Wilson,Daniel Wilson,Tulsa,OK,35,M,0 days 08:23:01,503,36.156312,-95.992752,"36.1563122, -95.9927516"
1,2,Eric,Davis,Eric Davis,Greenwood,IN,38,M,0 days 08:57:54,538,39.613699,-86.109543,"39.6136987, -86.1095429"


In [ ]:
df_race_results.to_csv('cleaned_dataset_for_race_results.csv',index= False)